#### [PyTorch를 활용한 머신러닝, 딥러닝 철저 입문]
## [Chapter 7-2] 예제: 손글씨 이미지 분류 (2)

---

## \#1. Import Modules

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_digits
from sklearn import datasets, model_selection

import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline

---

## \#2. Load Data

### 2-1. Get MNIST Data

In [2]:
mnist = datasets.fetch_openml('mnist_784', data_home='./data')
mnist.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'details', 'categories', 'url'])

#### mnist data 의 X (feature) 를 255로 정규화 해서 변수에 저장

In [3]:
mnist_data = mnist.data / 255
mnist_data.shape

(70000, 784)

#### mnist data의 y (label)을 변수에 저장
- mnist data가 업데이트 되면서, label의 target value들이 모두 `string` type의 데이터가 되었으므로 `int`로 형변환을 해준다.

In [4]:
mnist_label = mnist.target
mnist_label = mnist_label.astype(int)
print(mnist_label.shape)
mnist_label

(70000,)


array([5, 0, 4, ..., 4, 5, 6])

---

## \#3. Training / Test Data

### 3-1. Get Train / Test data by `train_test_split`

train data는 70000개의 데이터 중 5000개, test data는 500개를 추출하기로 한다.

In [5]:
train_size = 5000
test_size = 500

train_X, test_X, train_y, test_y = model_selection.train_test_split(mnist_data, mnist_label, \
                                                                    train_size = train_size, test_size=test_size)

print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

(5000, 784) (5000,)
(500, 784) (500,)


### 3-2. 2D Array for CNN

In [6]:
# reshape: (Data nums, channel nums, height, width)
train_X = train_X.reshape(len(train_X), 1, 28, 28)
test_X = test_X.reshape(len(test_X), 1, 28, 28)

---

## \#4. Make Tensor

### 4-1. Transform Data Type

#### 완성된 train, test 데이터를 PyTorch의 `tensor 데이터`로 변환한다.

In [7]:
# train data
train_X = torch.from_numpy(train_X).float()
train_y = torch.from_numpy(train_y).long()

# test data
test_X = torch.from_numpy(test_X).float()
test_y = torch.from_numpy(test_y).long()

train_X.shape, train_y.shape

(torch.Size([5000, 1, 28, 28]), torch.Size([5000]))

#### train의 X와 y데이터를 `tensorDataset` 으로 합친다.

In [18]:
train = TensorDataset(train_X, train_y)
train[0][0].shape, train[0][1]

(torch.Size([1, 28, 28]), tensor(2))

- train의 각 element들은 feature와 label의 쌍으로 이루어져있음을 확인
- 첫 번째 쌍은 `28 x 28`의 2차원 텐서와, `2`라는 label의 쌍으로 이루어져 있다.

### 4-2. Mini Batch

#### train 데이터를 미니배치로 학습시킬 수 있도록 100개 단위로 분할한다.

In [19]:
train_loader = DataLoader(train, batch_size=100, shuffle=True)

---

## \#5. Neural Network Composition

### 5-1. Neural Network class

In [21]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # 합성곱층
        self.conv1 = nn.Conv2d(1, 6, 5) #(입력 채널 수, 출력 채널 수, 필터크기)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # 완전연결층
        # 입력노드수 : ((28-5+1) / 2) -5+1) / 2
        self.func1 = nn.Linear(256, 64)
        self.func2 = nn.Linear(64, 10)
        
        
    def forward(self, x):
        # 입력층 - 합성곱1층
        x = F.relu(self.conv1(x))
        # 풀링1층
        x = F.max_pool2d(x, 2)
        # 합성곱2층
        x = F.relu(self.conv2(x))
        # 풀링2층
        x = F.max_pool2d(x, 2)
        
        # len = 256의 1차원 텐서로 풀어주기
        x = x.view(-1, 256)
        
        # 완전연결층
        x = F.relu(self.func1(x))
        
        # 출력층
        x = self.func2(x)
        # 출력층 활성화함수 : log_softmax
        y = F.log_softmax(x, dim=1)
        
        return y

### 5-2. NN Instance 생성

In [22]:
model = Net()
model

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (func1): Linear(in_features=256, out_features=64, bias=True)
  (func2): Linear(in_features=64, out_features=10, bias=True)
)

합성곱이 두 층 있고, 완전연결층과 출력층까지 확인

---

## \#6. Model Training

### 6-1. Loss Function

In [23]:
criterion = nn.CrossEntropyLoss()

### 6-2. Optimizer

In [24]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

### 6-3. Training

In [25]:
%%time
print("{} \t {}".format("epoch", "total loss"))
print("----- \t ----------------------")

for epoch in range(1000):
    total_loss = 0
    
    for train_X, train_y in train_loader:
        
        # 계산 그래프 구성
        train_X, train_y = Variable(train_X), Variable(train_y)
        
        # gradient 초기화
        optimizer.zero_grad()
        
        # forward 계산
        output = model.forward(train_X)
        
        # loss 계산
        loss = criterion(output, train_y)
        
        # 오차 역전파
        loss.backward()
        
        # gradient update
        optimizer.step()
        
        # 누적 오차 계산
        total_loss += loss.data
        
    # 50번째 epoch마다 loss 출력    
    if (epoch+1) % 100 == 0:
        print("{} \t {}".format(epoch+1, total_loss))

epoch 	 total loss
----- 	 ----------------------
100 	 4.132058143615723
200 	 0.9005931615829468
300 	 0.277071088552475
400 	 0.11689137667417526
500 	 0.06956642121076584
600 	 0.047022223472595215
700 	 0.03501998633146286
800 	 0.02728416956961155
900 	 0.022063668817281723
1000 	 0.018530182540416718
CPU times: user 16min 27s, sys: 6min 22s, total: 22min 49s
Wall time: 15min 54s


- 16분이나 걸렸다..

> 다음은 손글씨 이미지 분류 (1)의 결과다.

#### Chapter 6.3 손글씨이미지 분류 (1)
```
epoch 	 total loss
----- 	 ----------------------
100   	 34.955047607421875
200 	   4.097681999206543
300 	   0.3379775285720825
400 	   0.14065010845661163
500 	   0.060440998524427414
600 	   0.0877748653292656
700 	   0.04214557260274887
800 	   0.04590927064418793
900 	   0.035796478390693665
1000 	  0.019102295860648155  
```

- 최종적인 loss값은 크게 차이 나지 않는 것 같으나.. 초기 loss는 확연히 작았음을 확인

---

## \#7. Accuracy

### 7-1. test data 로 accuracy 계산

In [26]:
# 계산 그래프 구성
test_X, test_y = Variable(test_X), Variable(test_y)

# 결과를 0 또는 1이 되도록 변환
result = torch.max(model(test_X).data, 1)[1]

# test_y 와 결과가 같은 예측값의 개수 (맞춘 개수) 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

accuracy

0.966

92.6%의 정확도에 그쳤던 손글씨이미지분류(1)에서보다 약 4% 올랐음을 확인!